<a href="https://colab.research.google.com/github/Cactice/bert-ja-experiment/blob/master/first_attempt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --recurse-submodules https://github.com/yoheikikuta/bert-japanese

Cloning into 'bert-japanese'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 164 (delta 0), reused 2 (delta 0), pack-reused 160
Receiving objects: 100% (164/164), 218.44 KiB | 2.87 MiB/s, done.
Resolving deltas: 100% (87/87), done.
Submodule 'bert' (https://github.com/google-research/bert.git) registered for path 'bert'
Submodule 'wikiextractor' (https://github.com/attardi/wikiextractor.git) registered for path 'wikiextractor'
Cloning into '/content/bert-japanese/bert'...
remote: Enumerating objects: 340, done.        
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340        
Receiving objects: 100% (340/340), 310.70 KiB | 4.03 MiB/s, done.
Resolving deltas: 100% (186/186), done.
Cloning into '/content/bert-japanese/wikiextractor'...
remote: Enumerating objects: 613, done.        
remote: Total 613 (delta 0), reused 0 (delta 0), pack-reused 613        
Receiving objects: 100

In [2]:
pip install gdown

In [7]:
https://drive.google.com/file/d/11V3dT_xJUXsZRuDK1kXiXJRBSEHGl3In/view?usp=sharing
https://drive.google.com/file/d/1Otwik7_nNnOWyqGCgwJciBSwqaABLpXS/view?usp=sharing
https://drive.google.com/file/d/1F4b_u-5zzqabA6OfLxDkLh0lzqVIEZuN/view?usp=sharing
https://drive.google.com/file/d/1LB00MDQJjb-xLmgBMhdQE3wKDOLjgum-/view?usp=sharing
https://drive.google.com/file/d/1V9TIUn5wc-mB_wabYiz9ikvLsscONOKB/view?usp=sharing
https://drive.google.com/file/d/1jjZmgSo8C9xMIos8cUMhqJfNbyyqR0MY/view?usp=sharing
https://drive.google.com/file/d/1uzPpW38LcS4YS431GgdG0Hsj4gNgE5X1/view?usp=sharing

SyntaxError: ignored

In [0]:
!gdown https://drive.google.com/uc?id=11V3dT_xJUXsZRuDK1kXiXJRBSEHGl3In
!gdown https://drive.google.com/uc?id=1Otwik7_nNnOWyqGCgwJciBSwqaABLpXS
!gdown https://drive.google.com/uc?id=1F4b_u-5zzqabA6OfLxDkLh0lzqVIEZuN
!gdown https://drive.google.com/uc?id=1LB00MDQJjb-xLmgBMhdQE3wKDOLjgum-
!gdown https://drive.google.com/uc?id=1V9TIUn5wc-mB_wabYiz9ikvLsscONOKB
!gdown https://drive.google.com/uc?id=1jjZmgSo8C9xMIos8cUMhqJfNbyyqR0MY
!gdown https://drive.google.com/uc?id=1uzPpW38LcS4YS431GgdG0Hsj4gNgE5X1

Downloading...
From: https://drive.google.com/uc?id=11V3dT_xJUXsZRuDK1kXiXJRBSEHGl3In
To: /content/graph.pbtxt
11.0MB [00:00, 94.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Otwik7_nNnOWyqGCgwJciBSwqaABLpXS
To: /content/jawiki-20181220-pages-articles-multistream.xml.bz2
2.55GB [00:28, 120MB/s]

In [0]:
# find the share link of the file/folder on Google Drive
file_share_link = "https://drive.google.com/open?id=0B_URf9ZWjAW7SC11Xzc4R2d0N2c"

# extract the ID of the file
file_id = file_share_link[file_share_link.find("=") + 1:]

# append the id to this REST command
file_download_link = "https://docs.google.com/uc?export=download&id=" + file_id 